### ETL and Feature engineering
In the initial data exploration it was established that the data is skewed , has some outliers and missing values for numerical columns. There are categorical features that need to be one hot encoded. Since multiple models will be used multiple saves will be created depending on the transformations. For eg, Tree based models do not require that data be scaled while it is a good idea to scale data for neural networks.

In total 3 different ETL pipelines are created. One with one hot encoded categorical columns, one with scaled columns and one with scaled and transformed. The Yeo jhonson transformation is used on numerical columns to handle skewness. this should work well with Neural Network.

In [1]:
# Importing necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

In [2]:
# importing the dataset
df = pd.read_csv('online_shoppers_intention.csv')
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0.0,-1.0,0.0,-1.0,1.0,-1.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [17]:
# Clipping the upper and lower parts of the columns
df['Administrative'] = df['Administrative'].clip(lower=0, upper=20)
df['PageValues']=df['PageValues'].clip(lower=0,upper=250)
df['Administrative_Duration']=df['Administrative_Duration'].clip(lower=0,upper=1500)
df['Informational']=df['Informational'].clip(lower=0,upper=10)
df['ProductRelated_Duration']=df['ProductRelated_Duration'].clip(lower=0,upper=20000)
df['ProductRelated']=df['ProductRelated'].clip(lower=0,upper=400)
df['Informational_Duration']=df['Informational_Duration'].clip(lower=0,upper=1500)

In [18]:
# chechking month's dtype
df['Month'].dtypes

dtype('O')

In [19]:
# changing the datatype of the month's column
lis = ['Feb', 'Mar', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec']
cat_dtype = pd.api.types.CategoricalDtype(categories=lis, ordered=True)
df['Month']=df['Month'].astype(cat_dtype)

In [20]:
# chechking month's dtype again
df['Month'].dtypes

CategoricalDtype(categories=['Feb', 'Mar', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct',
                  'Nov', 'Dec'],
                 ordered=True)

In [21]:
# checking datatypes of columns in the dataset
df.dtypes

Administrative              float64
Administrative_Duration     float64
Informational               float64
Informational_Duration      float64
ProductRelated              float64
ProductRelated_Duration     float64
BounceRates                 float64
ExitRates                   float64
PageValues                  float64
SpecialDay                  float64
Month                      category
OperatingSystems              int64
Browser                       int64
Region                        int64
TrafficType                   int64
VisitorType                  object
Weekend                        bool
Revenue                        bool
dtype: object

In [23]:
# changing datatypes in categorical
cat_cols=['Weekend','VisitorType','Region','Browser','OperatingSystems','TrafficType']
df[cat_cols] = df[cat_cols].astype('category')
df = df.dropna()

In [27]:
df_encoded = pd.get_dummies(df,columns = cat_cols+['Month'], prefix = cat_cols+['Month'])
df_encoded.columns

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Revenue',
       'Weekend_False', 'Weekend_True', 'VisitorType_New_Visitor',
       'VisitorType_Other', 'VisitorType_Returning_Visitor', 'Region_1',
       'Region_2', 'Region_3', 'Region_4', 'Region_5', 'Region_6', 'Region_7',
       'Region_8', 'Region_9', 'Browser_1', 'Browser_2', 'Browser_3',
       'Browser_4', 'Browser_5', 'Browser_6', 'Browser_7', 'Browser_8',
       'Browser_9', 'Browser_10', 'Browser_11', 'Browser_12', 'Browser_13',
       'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3',
       'OperatingSystems_4', 'OperatingSystems_5', 'OperatingSystems_6',
       'OperatingSystems_7', 'OperatingSystems_8', 'TrafficType_1',
       'TrafficType_2', 'TrafficType_3', 'TrafficType_4', 'TrafficType_5',
       'TrafficType_6', 'TrafficType_7', 'TrafficType

In [29]:
numeric_cols = df.select_dtypes(include='float64').columns
numeric_cols

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay'],
      dtype='object')

In [30]:
# scaling the columns
scaler = MinMaxScaler()
pt=PowerTransformer()
df_encoded_transformed = df_encoded.copy()
df_encoded_scaled = df_encoded.copy()
df_encoded_scaled[numeric_cols]= scaler.fit_transform(df_encoded[numeric_cols])
df_encoded_transformed[numeric_cols]= pt.fit_transform(df_encoded[numeric_cols])

In [34]:
save_name='online_shoppers_intention_encoded.csv'
df_encoded.to_csv(save_name,index=False)
save_name='online_shoppers_intention_encoded_scaled.csv'
df_encoded_scaled.to_csv(save_name,index=False)
save_name='online_shoppers_intention_encoded_scaled_transformed.csv'
df_encoded_transformed.to_csv(save_name,index=False)

#### Finally the state of the scaler and transformer are saved so that it can be used to rescale if necessary.

In [35]:
import pickle

In [36]:
pickle.dump(scaler,open('Min_Max_scaler.pkl','wb'))
pickle.dump(pt,open('PowerTRansformer.pkl','wb'))

check the files if you want to make sure that they are saved

In [37]:
pd.read_csv('online_shoppers_intention_encoded.csv').head(1)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,0.0,0.0,0.0,0.0,1.0,0.0,0.2,0.2,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [38]:
pd.read_csv('online_shoppers_intention_encoded_scaled.csv').head(1)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,0.0,0.0,0.0,0.0,0.0025,0.0,1.0,1.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [39]:
pd.read_csv('online_shoppers_intention_encoded_scaled_transformed.csv').head(1)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Mar,Month_May,Month_June,Month_Jul,Month_Aug,Month_Sep,Month_Oct,Month_Nov,Month_Dec
0,-0.991065,-0.997544,-0.52074,-0.491916,-1.923024,-2.094361,2.207184,1.983626,-0.529787,-0.336232,...,1,0,0,0,0,0,0,0,0,0
